In [49]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator

In [23]:
# Load data
y = pd.read_csv("input/Ospiti.csv")
y["Data"] = pd.to_datetime(y["Data"]).dt.date
y 

,Data,Ospiti
0,2021-06-01,319.0
1,2021-06-02,396.0
2,2021-06-03,188.0
3,2021-06-04,258.0
4,2021-06-05,571.0
...,...,...
542,2022-11-26,732.0
543,2022-11-27,495.0
544,2022-11-28,226.0
545,2022-11-29,289.0


In [18]:
dates = pd.date_range(start=y["Data"].iloc[-1],periods=8)[1:]
dates

DatetimeIndex(['2022-12-01', '2022-12-02', '2022-12-03', '2022-12-04',
               '2022-12-05', '2022-12-06', '2022-12-07'],
              dtype='datetime64[ns]', freq='D')

In [26]:
y["Ospiti"].values

array([ 319.        ,  396.        ,  188.        ,  258.        ,
        571.        ,  488.        ,  214.        ,  259.        ,
        373.        ,  344.        ,  333.        ,  473.        ,
        352.        ,  220.        ,  338.        ,  197.        ,
        271.        ,  350.        ,  500.        ,  345.        ,
        217.        ,  271.        ,  334.        ,  336.        ,
        352.        ,  275.        ,  325.        ,  283.        ,
        317.        ,  292.        ,  285.        ,  218.        ,
        490.        ,  429.        ,  302.        ,  201.        ,
        349.        ,  400.        ,  355.        ,  469.        ,
        185.        ,  212.        ,  301.        ,  298.        ,
        348.        ,  316.        ,  582.        ,  306.        ,
        327.        ,  293.        ,  335.        ,  332.        ,
        308.        ,  469.        ,  435.        ,  282.        ,
        303.        ,  346.        ,  310.        ,  360.     

In [28]:
# normalize data with MinMaxScaler
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y["Ospiti"].values.reshape(-1,1))
y_scaled

array([[0.18949343],
       [0.26172608],
       [0.06660413],
       [0.13227017],
       [0.42589118],
       [0.34803002],
       [0.09099437],
       [0.13320826],
       [0.24015009],
       [0.21294559],
       [0.20262664],
       [0.33395872],
       [0.22045028],
       [0.09662289],
       [0.20731707],
       [0.0750469 ],
       [0.14446529],
       [0.21857411],
       [0.35928705],
       [0.21388368],
       [0.09380863],
       [0.14446529],
       [0.20356473],
       [0.2054409 ],
       [0.22045028],
       [0.14821764],
       [0.19512195],
       [0.15572233],
       [0.18761726],
       [0.1641651 ],
       [0.1575985 ],
       [0.09474672],
       [0.34990619],
       [0.29268293],
       [0.17354597],
       [0.07879925],
       [0.21763602],
       [0.26547842],
       [0.22326454],
       [0.33020638],
       [0.06378987],
       [0.0891182 ],
       [0.17260788],
       [0.16979362],
       [0.21669794],
       [0.18667917],
       [0.43621013],
       [0.177

In [45]:
# data windowing with a 30 days window
window_size = 30
window = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=y["Ospiti"],
    targets=None,
    sequence_length=window_size,
    sequence_stride=1,
    shuffle=True,
    batch_size=32,
)

In [46]:
# build a forecasting model using the Keras Sequential API
model = keras.Sequential(
    [
        layers.Dense(256, activation="relu", name="layer1"),
        layers.Dense(128, activation="relu", name="layer2"),
        layers.Dense(64, activation="relu", name="layer3"),
        layers.Dense(7, name="layer4"),
    ]
)

In [47]:
model.compile(optimizer='adam', loss="mse")

In [48]:
model.fit(window,epochs=100)

Epoch 1/100


ValueError: in user code:

    File "/home/davide/.local/lib/python3.8/site-packages/keras/engine/training.py", line 878, in train_function  *
        return step_function(self, iterator)
    File "/home/davide/.local/lib/python3.8/site-packages/keras/engine/training.py", line 867, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/davide/.local/lib/python3.8/site-packages/keras/engine/training.py", line 860, in run_step  **
        outputs = model.train_step(data)
    File "/home/davide/.local/lib/python3.8/site-packages/keras/engine/training.py", line 808, in train_step
        y_pred = self(x, training=True)
    File "/home/davide/.local/lib/python3.8/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/davide/.local/lib/python3.8/site-packages/keras/layers/core/dense.py", line 139, in build
        raise ValueError('The last dimension o